In [5]:
%pip install groq python-dotenv --quiet

In [6]:
from dotenv import load_dotenv
load_dotenv()

import os
import getpass
from groq import Groq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

client = Groq(api_key=os.environ["GROQ_API_KEY"])

BASE_MODEL = "llama-3.3-70b-versatile"
print(f"Client ready. Base model: {BASE_MODEL}")

Client ready. Base model: llama-3.3-70b-versatile


In [7]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": """You are a Senior Software Engineer and debugging expert.
Your responses are precise, code-focused, and rigorous.
Rules:
- Always identify the root cause before suggesting a fix.
- Provide working code snippets when relevant.
- Use correct technical terminology.
- If you need more info to debug, ask for the full traceback or stack trace.
- Keep explanations concise but technically complete.""",
        "temperature": 0.2,  # Low: precision matters more than creativity
        "description": "Bug reports, errors, code issues"
    },

    "billing": {
        "system_prompt": """You are a compassionate and professional Billing Support Specialist.
Your responses are empathetic, clear, and policy-driven.
Rules:
- Always acknowledge the customer's frustration first.
- Explain policies clearly without using jargon.
- Offer concrete next steps (e.g., 'I will escalate this to our billing team within 24 hours').
- Never promise refunds you cannot guarantee — say 'we will review' instead.
- End every response with a reassurance.""",
        "temperature": 0.5,
        "description": "Refunds, charges, subscriptions, payments"
    },

    "sales": {
        "system_prompt": """You are an enthusiastic and knowledgeable Sales Consultant.
Your responses are warm, persuasive, and benefit-focused.
Rules:
- Highlight the value and benefits of the product, not just features.
- Ask qualifying questions to understand what the customer actually needs.
- Mention relevant pricing tiers or trial options when applicable.
- Always end with a clear call-to-action.""",
        "temperature": 0.7,
        "description": "New inquiries, pricing, product questions"
    },

    "general": {
        "system_prompt": """You are a friendly and helpful general customer support assistant.
You handle casual questions, greetings, and anything that doesn't fit a specific category.
Rules:
- Be warm, concise, and helpful.
- If the question seems like it should go to a specialist (billing, technical, sales),
  gently suggest the right team.
- Keep responses short unless detail is specifically requested.""",
        "temperature": 0.7,
        "description": "Fallback for everything else"
    }
}

VALID_CATEGORIES = list(MODEL_CONFIG.keys())
print("Experts defined:", VALID_CATEGORIES)
for cat, cfg in MODEL_CONFIG.items():
    print(f"  [{cat}] -> {cfg['description']}")

Experts defined: ['technical', 'billing', 'sales', 'general']
  [technical] -> Bug reports, errors, code issues
  [billing] -> Refunds, charges, subscriptions, payments
  [sales] -> New inquiries, pricing, product questions
  [general] -> Fallback for everything else


In [9]:
import os
import getpass

# Force clear and re-enter
del os.environ["GROQ_API_KEY"]
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

from groq import Groq
client = Groq(api_key=os.environ["GROQ_API_KEY"])
print("Client ready.")

Enter your Groq API Key: ··········
Client ready.


In [10]:
ROUTER_SYSTEM_PROMPT = f"""You are a text classification engine. Your only job is to classify customer support queries.

Categories:
- technical: bug reports, error messages, code issues, crashes, API problems
- billing: charges, refunds, subscriptions, payments, invoices, pricing disputes
- sales: new product inquiries, demos, trial requests, feature comparisons, upgrades
- general: greetings, casual questions, feedback, anything else

CRITICAL RULE: You MUST respond with ONLY one word — the category name.
Do not add punctuation, explanation, or any other text.
Valid outputs: technical | billing | sales | general"""


def route_prompt(user_input: str) -> str:
    """
    Classifies the user's query into one of the expert categories.

    Args:
        user_input: The raw customer query.

    Returns:
        A category string: 'technical', 'billing', 'sales', or 'general'.
    """
    response = client.chat.completions.create(
        model=BASE_MODEL,
        messages=[
            {"role": "system", "content": ROUTER_SYSTEM_PROMPT},
            {"role": "user",   "content": user_input}
        ],
        temperature=0,      # Deterministic classification
        max_tokens=10,      # We only need one word
    )

    # Clean and validate the output
    raw = response.choices[0].message.content.strip().lower()

    # Strip any accidental punctuation
    category = raw.strip('.,!?"\'')

    # Fallback to 'general' if the model hallucinates an invalid category
    if category not in VALID_CATEGORIES:
        print(f"  [Router Warning] Got '{raw}', defaulting to 'general'")
        category = "general"

    return category


# Quick test
print("Router test:")
print(f"  'IndexError on line 5'  -> {route_prompt('IndexError on line 5')}")
print(f"  'charged twice'         -> {route_prompt('I was charged twice this month')}")
print(f"  'What plans do you have?'-> {route_prompt('What pricing plans do you offer?')}")
print(f"  'Hello!'                -> {route_prompt('Hello!')}")

Router test:
  'IndexError on line 5'  -> technical
  'charged twice'         -> billing
  'What plans do you have?'-> sales
  'Hello!'                -> general


In [11]:
import random
import re

# Mock price database — in production, replace with a real API call
MOCK_PRICES = {
    "bitcoin":  {"symbol": "BTC", "price": 67_432.18, "change_24h": +2.3},
    "ethereum": {"symbol": "ETH", "price":  3_521.44, "change_24h": -0.8},
    "solana":   {"symbol": "SOL", "price":    178.92, "change_24h": +5.1},
    "dogecoin": {"symbol": "DOGE","price":      0.14, "change_24h": +1.2},
}

def mock_fetch_price(query: str) -> str:
    """
    Mock tool: detects which crypto is mentioned and returns a price.
    In production: replace with requests.get('https://api.coingecko.com/...')
    """
    query_lower = query.lower()
    for coin, data in MOCK_PRICES.items():
        if coin in query_lower or data["symbol"].lower() in query_lower:
            direction = "^" if data["change_24h"] > 0 else "v"
            return (
                f"[TOOL RESULT — Mock Data]\n"
                f"{data['symbol']} current price: ${data['price']:,.2f} USD\n"
                f"24h change: {direction} {abs(data['change_24h'])}%"
            )
    return "[TOOL RESULT] Sorry, I don't have price data for that asset right now."


def is_price_query(user_input: str) -> bool:
    """Detect if the query is asking for a live price."""
    price_keywords = ["price", "cost", "worth", "value", "how much is", "current price"]
    crypto_keywords = list(MOCK_PRICES.keys()) + ["btc", "eth", "crypto", "coin"]

    lower = user_input.lower()
    has_price_kw  = any(kw in lower for kw in price_keywords)
    has_crypto_kw = any(kw in lower for kw in crypto_keywords)
    return has_price_kw and has_crypto_kw


# Test
print(is_price_query("What is the current price of Bitcoin?"))  # True
print(is_price_query("I have a billing issue"))                  # False
print(mock_fetch_price("current price of bitcoin"))

True
False
[TOOL RESULT — Mock Data]
BTC current price: $67,432.18 USD
24h change: ^ 2.3%


In [12]:
def process_request(user_input: str, verbose: bool = True) -> str:
    """
    Main orchestrator. Routes query to the correct expert and returns the response.

    Args:
        user_input: The raw customer query.
        verbose: If True, prints routing decisions for transparency.

    Returns:
        The expert's response as a string.
    """
    if verbose:
        print(f"\n{'='*60}")
        print(f"USER: {user_input}")
        print(f"{'='*60}")

    # --- BONUS: Tool Use Check (before routing) ---
    if is_price_query(user_input):
        if verbose:
            print("[Router] -> TOOL USE (price lookup — skipping LLM)")
        result = mock_fetch_price(user_input)
        if verbose:
            print(f"\n{result}")
        return result

    # --- Step 1: Route ---
    category = route_prompt(user_input)
    if verbose:
        print(f"[Router] -> '{category}' ({MODEL_CONFIG[category]['description']})")

    # --- Step 2: Select Expert Config ---
    expert = MODEL_CONFIG[category]

    # --- Step 3: Call Expert LLM ---
    response = client.chat.completions.create(
        model=BASE_MODEL,
        messages=[
            {"role": "system", "content": expert["system_prompt"]},
            {"role": "user",   "content": user_input}
        ],
        temperature=expert["temperature"],
        max_tokens=512,
    )

    answer = response.choices[0].message.content.strip()

    if verbose:
        print(f"\n[{category.upper()} EXPERT]:\n{answer}")

    return answer


print("Orchestrator ready.")

Orchestrator ready.


In [13]:
# Test 1: Technical query
process_request("My Python script is throwing an IndexError on line 5. The list has 3 items.");


USER: My Python script is throwing an IndexError on line 5. The list has 3 items.
[Router] -> 'technical' (Bug reports, errors, code issues)

[TECHNICAL EXPERT]:
To accurately diagnose the issue, I'll need more information. Can you please provide the full traceback or stack trace, as well as the relevant code snippet (at least lines 1-5) where the error is occurring?

Additionally, an `IndexError` typically occurs when you're trying to access an index that doesn't exist in the list. Given that the list has 3 items, valid indices would be 0, 1, and 2. Attempting to access index 3 or higher would result in an `IndexError`.

Here's a simple example of how an `IndexError` might occur:
```python
my_list = [1, 2, 3]
print(my_list[3])  # This would raise an IndexError
```
Please provide more context so I can better assist you in identifying the root cause and suggesting a fix.


In [14]:
# Test 2: Billing query
process_request("I was charged twice for my subscription this month and I want a refund.");


USER: I was charged twice for my subscription this month and I want a refund.
[Router] -> 'billing' (Refunds, charges, subscriptions, payments)

[BILLING EXPERT]:
I can imagine how frustrating it must be to see an unexpected charge on your account, especially when it's a duplicate payment. I'm so sorry you're going through this.

I want to assure you that I'm here to help resolve this issue. Our policy is to ensure that our customers are only charged for the services they receive, and it sounds like there may have been an error in this case. I'll do my best to explain what might have happened and what we can do to fix it.

To move forward, I'll need to review your account and investigate the duplicate charge. I will escalate this to our billing team within 24 hours, and they will look into the matter further. We will review your account and the charges in question to determine the best course of action.

Please know that we will do our best to resolve this issue as quickly as possible

In [15]:
# Test 3: Sales query
process_request("I'm interested in your product. What pricing plans do you offer for a team of 10?");


USER: I'm interested in your product. What pricing plans do you offer for a team of 10?
[Router] -> 'sales' (New inquiries, pricing, product questions)

[SALES EXPERT]:
We have several pricing plans that can cater to a team of your size. Before I dive into the specifics, can you tell me a bit more about how you envision our product fitting into your team's workflow? What specific challenges or goals do you hope to address with our solution?

That being said, our pricing plans are designed to be flexible and scalable. For a team of 10, I'd recommend exploring our Business Plan, which starts at $99 per month. This plan includes features such as advanced collaboration tools, priority support, and customized onboarding to ensure a seamless integration.

We also offer a Premium Plan, which starts at $199 per month, and includes additional features such as enhanced security, dedicated account management, and access to our premium support team.

If you're looking to try before you buy, we al

In [16]:
# Test 4: General / fallback
process_request("Hey, I just wanted to say your support team was super helpful last week!");


USER: Hey, I just wanted to say your support team was super helpful last week!
[Router] -> 'general' (Fallback for everything else)

[GENERAL EXPERT]:
That's great to hear. I'm glad our team could assist you and provide a positive experience. If you have any other questions or need help with anything else, feel free to ask. Otherwise, have a fantastic day!


In [17]:
# Test 5: BONUS — Tool Use (crypto price)
process_request("What is the current price of Bitcoin?");


USER: What is the current price of Bitcoin?
[Router] -> TOOL USE (price lookup — skipping LLM)

[TOOL RESULT — Mock Data]
BTC current price: $67,432.18 USD
24h change: ^ 2.3%


In [18]:
# Test 6: BONUS — Another crypto
process_request("How much is Ethereum worth right now?");


USER: How much is Ethereum worth right now?
[Router] -> TOOL USE (price lookup — skipping LLM)

[TOOL RESULT — Mock Data]
ETH current price: $3,521.44 USD
24h change: v 0.8%


## 7. Batch Test — See the Router in Action

Run multiple queries at once and show only the routing decisions (no full responses) to verify the router is classifying correctly.

In [19]:
test_queries = [
    ("My API keeps returning a 429 rate limit error.",              "technical"),
    ("Can I get a receipt for my last payment?",                    "billing"),
    ("Do you offer a free trial?",                                  "sales"),
    ("What are your business hours?",                               "general"),
    ("The app crashes every time I click the export button.",       "technical"),
    ("I cancelled but was still charged for next month.",           "billing"),
    ("What's the difference between your Starter and Pro plans?",   "sales"),
    ("What is the price of Solana?",                                "tool_use"),
]

print(f"{'Query':<55} {'Expected':<12} {'Got':<12} {'✓?'}")
print("-" * 90)

for query, expected in test_queries:
    if expected == "tool_use":
        got = "tool_use" if is_price_query(query) else route_prompt(query)
    else:
        got = route_prompt(query)

    match = "✅" if got == expected else "❌"
    print(f"{query[:54]:<55} {expected:<12} {got:<12} {match}")

Query                                                   Expected     Got          ✓?
------------------------------------------------------------------------------------------
My API keeps returning a 429 rate limit error.          technical    technical    ✅
Can I get a receipt for my last payment?                billing      billing      ✅
Do you offer a free trial?                              sales        sales        ✅
What are your business hours?                           general      general      ✅
The app crashes every time I click the export button.   technical    technical    ✅
I cancelled but was still charged for next month.       billing      billing      ✅
What's the difference between your Starter and Pro pla  sales        sales        ✅
What is the price of Solana?                            tool_use     tool_use     ✅
